# Classification de textes avec FastText

Dans ce notebook, nous allons explorer la classification automatique de textes en utilisant FastText, une bibliothèque développée par Facebook Research. FastText permet d'entraîner des modèles de classification de texte de manière efficace et rapide, en s'appuyant sur des représentations de mots sous forme de vecteurs (word embeddings).

Nous verrons comment :
- Préparer les données textuelles pour FastText
- Entraîner un modèle de classification 
- Évaluer ses performances
- Utiliser le modèle pour faire des prédictions


In [36]:
# Importation des bibliothèques nécessaires
import fasttext
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Chargement d'un dataset plus complet depuis un fichier texte
def load_dataset(filename):
    textes = []
    labels = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            # Le format attendu est: label\ttext
            label, text = line.strip().split('\t')
            textes.append(text)
            labels.append(label)
    return pd.DataFrame({
        'texte': textes,
        'sentiment': labels
    })

# Chargement des données
df = load_dataset('data/critiques.txt')

print(f"Nombre total d'exemples: {len(df)}")
print("\nDistribution des labels:")
print(df['sentiment'].value_counts())

# Préparation des données au format FastText
def prepare_fasttext_format(text, label):
    return f"__label__{label} {text}"

# Création des fichiers d'entraînement et de test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Sauvegarde des données dans des fichiers
with open('data/train.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(train_data['texte'], train_data['sentiment']):
        f.write(prepare_fasttext_format(text, label) + '\n')
        
with open('data/test.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(test_data['texte'], test_data['sentiment']):
        f.write(prepare_fasttext_format(text, label) + '\n')

# Entraînement du modèle FastText
model = fasttext.train_supervised(input='data/train.txt', lr=1.0, epoch=25)

# Évaluation du modèle
print("\nÉvaluation du modèle:")
print(model.test('data/test.txt'))

# Exemple de prédiction
exemple_texte = "Ce produit est tout bonnement remarquable !"
prediction = model.predict(exemple_texte)
print(f"\nPrédiction pour '{exemple_texte}':")
print(f"Label: {prediction[0][0].replace('__label__', '')}")
print(f"Confiance: {prediction[1][0]:.2f}")

Nombre total d'exemples: 482

Distribution des labels:
sentiment
__label__positive    241
__label__negative    241
Name: count, dtype: int64

Évaluation du modèle:
(97, 0.9175257731958762, 0.9175257731958762)

Prédiction pour 'Ce produit est tout bonnement remarquable !':
Label: positive
Confiance: 0.99
